#0. OBTENER MASTER.JSON

Se debe abrir SCRIPT de video VIMEO y obtener enlace master.json completo, si posee .mp4 no es necesario este proceso

#1. DESCARGA DE ARCHIVOS

Se ingresa el parametro 
MASTER JSON, procede a descargar el audio y video en su mejor calidad (solo unos 2 minutos)

In [ ]:
#@title
#!/bin/env python3

#import argparse
import subprocess
import requests

#    master_json_url = "https://91vod-adaptive.akamaized.net" \
#    "/exp=1668939686" \
#    "~acl=%2F842360cb-7711-4e48-9bd5-144e4686a2e2%2F%2A" \
#    "~hmac=a982f9ed2cc1e2fc7c61201f9997e0262917b13a89494c4169e4b3848d148f59/842360cb-7711-4e48-9bd5-144e4686a2e2" \
#    "/sep" \
#    "/video/e524c67d,336fea7d,5e87cb6d,306933cc,d42a911f" \
#    "/audio/ebb79ef9,7c03de6a,cad3266d" \
#    "/master.json?query_string_ranges=1&base64_init=1" \

#master_json_url = "https://91vod-adaptive.akamaized.net/exp=1668995241~acl=%2F842360cb-7711-4e48-9bd5-144e4686a2e2%2F%2A~hmac=9d08db3017c9c54bb1732988a82f58f114024eb14419590f5108bfe05e7d36a6/842360cb-7711-4e48-9bd5-144e4686a2e2/sep/video/e524c67d,336fea7d,5e87cb6d,306933cc,d42a911f/audio/ebb79ef9,7c03de6a,cad3266d/master.json?query_string_ranges=1&base64_init=1"
#output_file = "/content/drive/MyDrive/DINTA/Clase21/result.mp4"  # "./result.mp4"


#@title Example form fields
#@markdown Forms support many types of fields.
master_json_url = "https://91vod-adaptive.akamaized.net/exp=1668995241~acl=%2F842360cb-7711-4e48-9bd5-144e4686a2e2%2F%2A~hmac=9d08db3017c9c54bb1732988a82f58f114024eb14419590f5108bfe05e7d36a6/842360cb-7711-4e48-9bd5-144e4686a2e2/sep/video/e524c67d,336fea7d,5e87cb6d,306933cc,d42a911f/audio/ebb79ef9,7c03de6a,cad3266d/master.json?query_string_ranges=1&base64_init=1" #@param {type: "string"}
#@markdown ---



#if len(sys.argv) > 1:
#     master_json_url = sys.argv[1]
#     output_file = sys.argv[2]
#else:
#    print('master.json must be passed as argument', file=sys.stderr)
#    exit(1)

# eliminanos de /SEP/ en adelante y agregamos /PARCEL/
base_url = master_json_url[:master_json_url.rfind('/sep/')]
#base_url = "https://117vod-adaptive.akamaized.net/exp=1668959455~acl=%2Fa78da094-44ed-4dc3-980a-19095f3b9acc%2F%2A~hmac=abf8ebfffc8c80493c37fd6cef039d13fd7371e53e72db5ba133abe20ba619f2/a78da094-44ed-4dc3-980a-19095f3b9acc/"
base_url = base_url + "/parcel/"

# obtenemos el MASTER.JSON
resp = requests.get(master_json_url)
content = resp.json()

# obtenemos la calidad mas alta de video
heights = [(i, d['height']) for (i, d) in enumerate(content['video'])]
idx, _ = max(heights, key=lambda h: h[1])
video = content['video'][idx]
video_base_url = base_url + video['base_url']
video_file =  video['id'] + '.mp4'
video_url = video_base_url + video_file
print('VIDEO URL:', video_url)

# Audio download here
bitrate = [(i, d['bitrate']) for (i, d) in enumerate(content['audio'])]

print('Bitrate', bitrate)

idx, _ = max(bitrate, key=lambda x: x[1])
audio = content['audio'][idx]
audio_base_url = base_url + audio['base_url']
audio_file = audio['id'] + '.mp4'
audio_url = video_base_url + audio_file
print('AUDIO URL:', audio_url)

print("Downloading Files ...")
with open("files.txt","w") as file:
    file.write(video_file+"\n")
    file.write(audio_file+"\n")

with open("urls.txt","w") as file:
    file.write(video_url+"\n")
    file.write(audio_url+"\n")

%time !wget -i urls.txt
print("Download Files!!!")

print( audio_file )
print( video_file )

#2. MONTAR GDRIVE

Se monta google Drive, para que el archivo resultante, quede disponible aun despues de la sesion Colab

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/drive')

#3. MIX AUDIO/VIDEO EN MP4

Se ejecuta subproceso para unir ambos archivos, puede tardar horas
el resultado lo dejara en el DRIVE (1 hora por Giga)

In [ ]:
#@title Example form fields
#@markdown Forms support many types of fields.
output_file = "/content/drive/MyDrive/DINTA/Class/result.mp4"  #@param {type: "string"}
#@markdown ---


#@title
# combinamos el audio y video en result.mp4
print('Combining video and audio...')
cmd = 'ffmpeg -y -stats -i '
cmd += audio_file
cmd += ' -i '
cmd += video_file
cmd += ' ' + output_file
%time subprocess.call(cmd, shell=True)
print('Mixing Done!')

# Borrar los archivos 
#!rm -f `cat files.txt`
print("Temporary files removed!")

print("Done!")

#4. AUTH GOOGLE

1. Se otorga permiso para acceder a Youtube y poder subir archivos, se requiere haber subido el archivo client_secret.json
1. Se debe abrir link, aceptar condiciones, copiar codigo y pegar en consola


In [ ]:
#!/usr/bin/python

#en console.cloud.google.com
# habilitar Credenciales Pantalla inicio
# Habilitar OAuth 2.0 modo Desktop
# descargar secret_client.json
# Habilitar la API especifica
#https://console.developers.google.com/apis/api/youtube.googleapis.com/overview?project=655932566158

#desde la APP cliente
# usando el client_secret.json, SCPE, AP y API version
# entregara LINK para habilitar permiso
# entregara un codigo para pegar en el cliente


import httplib2
import os
import random
import time

import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow


#@title Example form fields
#@markdown Forms support many types of fields.
CLIENT_SECRETS_FILE = "/content/drive/MyDrive/UTILS/VIMEO_MIX_UPLOAD_YT/client_secret.json" #@param {type: "string"}
#@markdown ---


SCOPES = ['https://www.googleapis.com/auth/youtube.upload']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

# Authorize the request and store authorization credentials.
def get_authenticated_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

youtube = get_authenticated_service()

print('YT KEY:')
print(youtube)

#5. SUBIR A YOUTUBE

Realiza proceso de carga del archivo mp4 resultante, almacenado en google Drive
(solo unos 30 segundos)

In [ ]:
#@title Example form fields
#@markdown Forms support many types of fields.
TITULO_VIDEO = "Video de Prueba" #@param {type: "string"}
#@markdown ---


# previamente definido
MEDIA_FILE_PATH = output_file

httplib2.RETRIES = 1
MAX_RETRIES = 10
RETRIABLE_EXCEPTIONS = (httplib2.HttpLib2Error, IOError)
RETRIABLE_STATUS_CODES = [500, 502, 503, 504]
VALID_PRIVACY_STATUSES = ('public', 'private', 'unlisted')

def initialize_upload(youtube, title_video):

  options_title = title_video
  options_description ='Video description'
  options_category ='22'
  options_privacyStatus ='unlisted'

  body=dict(
    snippet=dict(
      title=options_title,
      description=options_description,
      categoryId=options_category
    ),
    status=dict(
      privacyStatus=options_privacyStatus
    )
  )

  insert_request = youtube.videos().insert(
    part=','.join(body.keys()),
    body=body,
    media_body=MediaFileUpload(MEDIA_FILE_PATH, chunksize=-1, resumable=True)
  )

  resumable_upload(insert_request)

def resumable_upload(request):
  response = None
  error = None
  retry = 0
  while response is None:
    try:
      print ('Uploading file...')
      status, response = request.next_chunk()
      if response is not None:
        if 'id' in response:
          print ('Video id "%s" was successfully uploaded.' % response['id'])
        else:
          exit('The upload failed with an unexpected response: %s' % response)
    except HttpError as e:
      if e.resp.status in RETRIABLE_STATUS_CODES:
        error = 'A retriable HTTP error %d occurred:\n%s' % (e.resp.status, e.content)
      else:
        raise
    except RETRIABLE_EXCEPTIONS as e:
      error = 'A retriable error occurred: %s' % e

    if error is not None:
      print (error)
      retry += 1
      if retry > MAX_RETRIES:
        exit('No longer attempting to retry.')

      max_sleep = 2 ** retry
      sleep_seconds = random.random() * max_sleep
      print ('Sleeping %f seconds and then retrying...' % sleep_seconds)
      time.sleep(sleep_seconds)

def uploadYT():
# paso previamente realizado
#  youtube = get_authenticated_service()

  try:
    initialize_upload(youtube, TITULO_VIDEO)
  except HttpError as e:
    print ('An HTTP error %d occurred:\n%s' % e.resp.status, e.content)

uploadYT()

#6. PROPIEDADES DEL VIDEO EN YOUTUBE

Mientras Youtube *Procesa el video*, se pueden configurar caracteristicas tales como la Descripcion, la Lista, etc. se deben realizar directamente en Youtube Studio